In [1]:
from inaFaceAnalyzer import VideoAnalyzer
from inaFaceAnalyzer.face_detector import LibFaceDetection
#from inaFaceAnalyzer.opencv_utils import get_video_properties, analysisFPS2subsamp_coeff
from inaFaceAnalyzer.display_utils import ass_subtitle_export
import os
#import pandas as pd

In [2]:
gv = VideoAnalyzer(face_detector=LibFaceDetection())

2021-11-07 18:13:39.729185: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-07 18:13:39.730616: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-07 18:13:39.730852: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-07 18:13:39.731179: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:
analysis_fps = 5
vid_src = './media/pexels-artem-podrez-5725953.mp4'
%time ret = gv(vid_src, fps=analysis_fps)

2021-11-07 18:13:45.787614: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8204
2021-11-07 18:13:45.791754: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-11-07 18:13:46.898793: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.30GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-11-07 18:13:46.898865: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.30GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-11-07 18:13:46.952299: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.4

CPU times: user 10.2 s, sys: 526 ms, total: 10.7 s
Wall time: 8.33 s


In [4]:
ret

,frame,bbox,detect_conf,sex_decfunc,age_decfunc,sex_label,age_label
0,0,"(945, -16, 1138, 176)",0.999999,8.180833,3.883811,m,33.838105
1,0,"(70, 118, 272, 320)",0.999996,-12.277652,3.165739,f,26.657391
2,0,"(310, 163, 491, 343)",0.999979,-10.850051,2.996131,f,24.961309
3,0,"(558, 201, 727, 371)",0.999974,8.937375,2.911818,m,24.118176
4,0,"(744, 22, 929, 207)",0.981539,-7.614790,2.310285,f,18.102851
...,...,...,...,...,...,...,...
219,348,"(343, 130, 472, 259)",0.999986,-8.917944,2.868898,f,23.688979
220,348,"(-44, 108, 165, 319)",0.999736,-7.542466,3.348511,f,28.485112
221,354,"(877, 65, 1013, 201)",0.999999,-8.295115,2.623810,f,21.238101
222,354,"(611, 260, 796, 444)",0.999990,12.288671,2.911372,m,24.113717


In [5]:
dst = './mysubtitle.ass'
ass_subtitle_export(vid_src, ret, dst, analysis_fps)

In [6]:
os.system('vlc %s --sub-file %s' % (vid_src, dst))

[000055cad6f81670] main libvlc: Running vlc with the default interface. Use 'cvlc' to use vlc without interface.
[ass] Shaper: FriBidi 0.19.7 (SIMPLE) HarfBuzz-ng 2.6.4 (COMPLEX)
[ass] Using font provider fontconfig
libva info: VA-API version 1.7.0
libva info: Trying to open /usr/lib/x86_64-linux-gnu/dri/nvidia_drv_video.so
libva info: va_openDriver() returns -1
[00007f1bf0003880] glconv_vaapi_x11 gl error: vaInitialize: unknown libva error
libva info: VA-API version 1.7.0
libva info: Trying to open /usr/lib/x86_64-linux-gnu/dri/iHD_drv_video.so
libva info: Found init function __vaDriverInit_1_7
libva info: va_openDriver() returns 0
[00007f1bf8060a30] main video output error: video output creation failed
[00007f1c0cc16c90] main decoder error: failed to create video output
[00007f1c0cc16c90] avcodec decoder: Using NVIDIA VDPAU Driver Shared Library  470.74  Mon Sep 13 22:58:37 UTC 2021 for hardware decoding
[ass] fontselect: (DejaVu Sans, 400, 0) -> /usr/share/fonts/truetype/dejavu/Deja

0